In [1]:
! mkdir encoder
! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
  
! mkdir GloVe
! curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip GloVe/glove.840B.300d.zip -d GloVe/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  24.4M      0  0:00:06  0:00:06 --:--:-- 33.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   353    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 2075M  100 2075M    0     0  4968k      0  0:07:07  0:07:07 --:--:-- 4894k:21  0:22:12 2414k   0  0:16:40  0:00:27  0:16:13 4397k00  0:00:41  0:10:19 5249k  0  3674k      0  0:09:38  0:00:53  0:08:45 5245k:09:11  0:01:00  0:08:11 5244k0:06:42 5223k0     0  4493k      0  0:07:53  0:01:55  0:05:58 5239k 4515k      0  0:07:50  0:01:59  0:05:51 5148k:14 4786k4735k      0  0:07:28  0:03:35  0:03:53 5228k07:28  0:03:36  0:03:52 5227k9k      0  0

In [2]:
sentences = ["I ate dinner.", 
       "We had a three-course meal.", 
       "Brad came to dinner with us.",
       "He loves fish tacos.",
       "In the end, we all felt like we ate too much.",
       "We all agreed; it was a magnificent evening."]
import numpy as np
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [9]:
from models import InferSent
import torch

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = './GloVe/glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [10]:
model.build_vocab(sentences, tokenize=True)

Found 36(/36) words with w2v vectors
Vocab size : 36


In [11]:
query = "I had pizza and pasta"
query_vec = model.encode(query)[0]
query_vec

/Users/varunsatheesh/capstone/encoders/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


array([ 0.02459561,  0.04943121, -0.15705208, ...,  0.07534432,
       -0.039418  ,  0.05388856], dtype=float32)

In [12]:
similarity = []
for sent in sentences:
  sim = cosine(query_vec, model.encode([sent])[0])
  print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  I ate dinner. ; similarity =  0.686888
Sentence =  We had a three-course meal. ; similarity =  0.50432706
Sentence =  Brad came to dinner with us. ; similarity =  0.55740434
Sentence =  He loves fish tacos. ; similarity =  0.59071404
Sentence =  In the end, we all felt like we ate too much. ; similarity =  0.57681185
Sentence =  We all agreed; it was a magnificent evening. ; similarity =  0.50499654
